# Set up some ENV vars and configs

In [5]:
import sys
!{sys.executable} -m pip install ragas
!{sys.executable} -m pip install boto3
!{sys.executable} -m pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.4 MB/s eta 0:00:00a 0:00:01


In [3]:
import os
import langchain
import nest_asyncio

# langchain.debug=False
langchain.debug=True
os.environ["OPENAI_API_KEY"] = input("Open AI API Key")
os.environ["RAGAS_DO_NOT_TRACK "] = "true"

# NOTES: Only used when running on a jupyter notebook, otherwise comment or remove this function.
nest_asyncio.apply()

# Pull down a predefined question-answer dataset

In [6]:
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2", trust_remote_code=True)
print(amnesty_qa)
amnesty_qa['eval'].to_pandas()

DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})


/home/grey/code/sai/bedrock-ragas-poc/.venv/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[- Fossil fuel companies, whether state or pri..."
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The private companies responsible for the mos...
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,[Amnesty International called on its vast netw...
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,[Amnesty International recommends that the Spe...
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,[Amnesty International has therefore created t...
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[26. The Act raises serious questions about it...
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,[- The victims of gross human rights violation...
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...


## Example of pulling in our own local dataset

Unsure of why it labels this as "train" data, but I could not find a good way to rename it during data load.  It does not affect functionality as far as I can tell.

In [2]:
dataset = load_dataset("./data")
print(dataset)
dataset['train'].to_pandas()

DatasetDict({
    train: Dataset({
        features: ['ground_truth', 'question', 'answer', 'contexts'],
        num_rows: 2
    })
})


,ground_truth,question,answer,contexts
0,Your name is Grey,What is my name?,Your name is Grey.,[My name is Grey]
1,Your name is Grey,What is my name?,Your name is Matt.,[My name is Grey]


# Run using the default OpenAI embeddings and LLM

Note, we are only currently targeting the `answer_correctness` metric.  There are a few others in the documentation, but when we start looking at using bedrock instead of OpenAI, we start running in to issues.  `answer_correctness` is a nice hybrid metric that can capture a rough approximation of how your solution is doing.

In [4]:
from ragas import evaluate
from ragas.metrics import (
    # context_precision,
    # answer_relevancy,
    # faithfulness,
    # context_recall,
    answer_correctness
)

In [5]:
results_openai = evaluate(
    amnesty_qa['eval'],
    metrics=[
        answer_correctness
    ]
)

[chain/start] [1:chain:ragas evaluation] Entering Chain run with input:
{}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0] Entering Chain run with input:
{
  "question": "What are the global implications of the USA Supreme Court ruling on abortion?",
  "ground_truth": "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion gui

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

[llm/end] [1:chain:ragas evaluation > 8:chain:row 6 > 9:chain:answer_correctness > 10:llm:ChatOpenAI] [9.38s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "[{\"statements that are present in both the answer and the ground truth\": [\"The right that guarantees access to comprehensive information about past human rights violations, including the identities of the perpetrators and the fate of the victims, as well as the circumstances surrounding the violations, is the right to truth\"], \"statements present in the answer but not found in the ground truth\": [], \"relevant statements found in the ground truth but omitted in the answer\": [\"The right to know the truth\"]}]",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
           

Evaluating:  20%|██        | 4/20 [00:18<00:55,  3.44s/it]

[chain/end] [1:chain:ragas evaluation > 14:chain:row 12 > 15:chain:answer_correctness > 17:chain:answer_similarity] [8.37s] Exiting Chain run with output:
{
  "output": 0.926329893012103
}
[chain/end] [1:chain:ragas evaluation > 14:chain:row 12 > 15:chain:answer_correctness] [18.05s] Exiting Chain run with output:
{
  "output": 0.23158247325302575
}
[llm/end] [1:chain:ragas evaluation > 17:chain:row 15 > 18:chain:answer_correctness > 19:llm:ChatOpenAI] [18.05s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "[{\"statements that are present in both the answer and the ground truth\": [], \"statements present in the answer but not found in the ground truth\": [\"Criminalizing abortion can have severe consequences for marginalized individuals\", \"particularly those who are already facing various forms of discrimination and inequality\", \"Some of the consequences include:\", \"Health risks: When abortion is criminalized, individuals may resort to unsafe an

...

[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 6:chain:answer_similarity] [8.46s] Exiting Chain run with output:
{
  "output": 0.941819866108623
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness] [22.36s] Exiting Chain run with output:
{
  "output": 0.29795496652715575
}


Evaluating:  95%|█████████▌| 19/20 [00:22<00:00,  2.18it/s]

[chain/end] [1:chain:ragas evaluation > 6:chain:row 4 > 7:chain:answer_correctness > 9:chain:answer_similarity] [8.38s] Exiting Chain run with output:
{
  "output": 0.9191610139836923
}
[chain/end] [1:chain:ragas evaluation > 6:chain:row 4 > 7:chain:answer_correctness] [22.59s] Exiting Chain run with output:
{
  "output": 0.3235402534959231
}


Evaluating: 100%|██████████| 20/20 [00:23<00:00,  1.17s/it]

[chain/end] [1:chain:ragas evaluation > 19:chain:row 17 > 20:chain:answer_correctness > 22:chain:answer_similarity] [8.78s] Exiting Chain run with output:
{
  "output": 0.9565461441538132
}
[chain/end] [1:chain:ragas evaluation > 19:chain:row 17 > 20:chain:answer_correctness] [23.33s] Exiting Chain run with output:
{
  "output": 0.3073183542202715
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0] [23.40s] Exiting Chain run with output:
{
  "answer_correctness": 0.30503639425718665
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1] [23.40s] Exiting Chain run with output:
{
  "answer_correctness": 0.29795496652715575
}
[chain/end] [1:chain:ragas evaluation > 4:chain:row 2] [23.40s] Exiting Chain run with output:
{
  "answer_correctness": 0.5859766316305263
}
[chain/end] [1:chain:ragas evaluation > 5:chain:row 3] [23.40s] Exiting Chain run with output:
{
  "answer_correctness": 0.5317271404658686
}
[chain/end] [1:chain:ragas evaluation > 6:chain:row 4] [23.40s] Exiting Chain 

In [6]:
# Using Open AI
print(results_openai)
results_openai.to_pandas()

{'answer_correctness': 0.4459}


,question,ground_truth,answer,contexts,answer_correctness
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ...",0.305036
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[- Fossil fuel companies, whether state or pri...",0.297955
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The private companies responsible for the mos...,0.585977
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,[Amnesty International called on its vast netw...,0.531727
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,[Amnesty International recommends that the Spe...,0.323540
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,[Amnesty International has therefore created t...,0.746764
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[26. The Act raises serious questions about it...,0.748469
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,[- The victims of gross human rights violation...,0.485679
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,0.227985
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,0.537945


In [7]:
results_openai_custom_dataset = evaluate(
    dataset['train'],
    metrics=[
        answer_correctness
    ]
)

[chain/start] [1:chain:ragas evaluation] Entering Chain run with input:
{}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Grey.",
  "contexts": [
    "My name is Grey"
  ]
}
[chain/start] [1:chain:ragas evaluation > 3:chain:row 1] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Matt.",
  "contexts": [
    "My name is Grey"
  ]
}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Grey.",
  "contexts": [
    "My name is Grey"
  ]
}
[llm/start] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Extract foll

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

[llm/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:ChatOpenAI] [9.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "[{\"statements that are present in both the answer and the ground truth\": [\"Your name is Grey\"], \"statements present in the answer but not found in the ground truth\": [], \"relevant statements found in the ground truth but omitted in the answer\": []}]",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "[{\"statements that are present in both the answer and the ground truth\": [\"Your name is Grey\"], \"statements present in the answer but not found in the ground truth\

Evaluating:  50%|█████     | 1/2 [00:17<00:17, 17.81s/it]

[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 5:chain:answer_similarity] [8.82s] Exiting Chain run with output:
{
  "output": 0.9863957706507789
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] [17.82s] Exiting Chain run with output:
{
  "output": 0.9965989426626947
}


Evaluating: 100%|██████████| 2/2 [00:18<00:00,  9.24s/it]

[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 6:chain:answer_similarity] [8.49s] Exiting Chain run with output:
{
  "output": 0.8639679108573589
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness] [18.49s] Exiting Chain run with output:
{
  "output": 0.21599197771433973
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0] [18.49s] Exiting Chain run with output:
{
  "answer_correctness": 0.9965989426626947
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1] [18.49s] Exiting Chain run with output:
{
  "answer_correctness": 0.21599197771433973
}
[chain/end] [1:chain:ragas evaluation] [18.49s] Exiting Chain run with output:
{
  "answer_correctness": 0.6062954601885172
}


In [8]:
# Using Open AI
print(results_openai_custom_dataset)
results_openai_custom_dataset.to_pandas()

{'answer_correctness': 0.6063}


,ground_truth,question,answer,contexts,answer_correctness
0,Your name is Grey,What is my name?,Your name is Grey.,[My name is Grey],0.996599
1,Your name is Grey,What is my name?,Your name is Matt.,[My name is Grey],0.215992


# Running with Bedrock instead of OpenAI

We can run with Bedrock, but I have found the prompts being used for the different metrics do not perform well out of the box against many of the Bedrock models.  I have tried against Llama-2, ClaudeV2.1, and Claude Instant.  I have found the best results with Claude Instant, both before and after some of the changes we will be making.

We must first set up our connection to bedrock using boto3.  In this case, I am using an sso session I have named "dev".

In [11]:
import boto3

from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings

boto3.setup_default_session(profile_name="dev")
BEDROCK_CLIENT = boto3.client("bedrock-runtime", "us-east-1")
embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", client=BEDROCK_CLIENT
)
llm = BedrockChat(
    # model_id="anthropic.claude-v2:1",
    model_id="anthropic.claude-instant-v1",
    # model_id="meta.llama2-13b-chat-v1",
    model_kwargs={"temperature": 0.1, "max_tokens_to_sample": 1000}, # For Claude
    # model_kwargs={"temperature": 0.1, "max_gen_len": 1000}, # For Llama
    client=BEDROCK_CLIENT,
)

### If you would like to see if not perform well, you may uncomment the following and run it BEFORE you run any of the cells below.  Some of the cells below are destructive, and will change the way this cell will behave if you run it after.

Running "poorly" can mean a few different things.  In many of the cases I tried, it meant the LLM gave back responses that were not in JSON format, and the follow up prompts RAGAS used to try and get it to format them correctly were unsuccessful.  When this happened, that record in the dataset ends up with `Nan` instead of a score.

In the case of Claude Instant, the "badness" is much less pronounced. It USUALLY returns valid json with the original prompt, but not always.  Also, sometimes it responds to the example info instead of the last passed in example that we are targeting.  This leads to incorrect scores.  This is hard to see in the full dataset, but easier to see in the custom dataset, which should have 1 exact match for an answer, and one incorrect answer, resulting in around a .5 score.

Other models, like ClaudeV2 and Llama2 both end up returning mostly NaN records, or failing to complete entirely, unless we use the workarounds below.

In [12]:
results_bedrock_bad = evaluate(
    amnesty_qa['eval'],
    llm=llm,
    embeddings=embeddings,
    metrics=[
        answer_correctness
    ]
)

[chain/start] [1:chain:ragas evaluation] Entering Chain run with input:
{}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0] Entering Chain run with input:
{
  "question": "What are the global implications of the USA Supreme Court ruling on abortion?",
  "ground_truth": "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion gui

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

[llm/end] [1:chain:ragas evaluation > 4:chain:row 2 > 5:chain:answer_correctness > 6:llm:BedrockChat] [30.03s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are the extracted statements from the given question, answer and ground truth:\n\n{\"statements that are present in both the answer and the ground truth\": [\"ExxonMobil\", \"Chevron\", \"Peabody\"], \n\n\"statements present in the answer but not found in the ground truth\": [\"ConocoPhillips Company\", \"BP plc\", \"Royal Dutch Shell plc\", \"Duke Energy Corporation\", \"TotalEnergies SE\", \"BHP Group Limited\", \"Rio Tinto Group\", \"The rankings may change over time as new data becomes available.\"], \n\n\"relevant statements found in the ground truth but omitted in the answer\": []}",
        "generation_info": null,
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",


Evaluating:   5%|▌         | 1/20 [00:30<09:41, 30.61s/it]

[llm/end] [1:chain:ragas evaluation > 15:chain:row 13 > 16:chain:answer_correctness > 17:llm:BedrockChat] [30.49s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are the extracted statements from the given question, answer and ground truth:\n\n[\n  {\n    \"statements that are present in both the answer and the ground truth\": [\"The purpose of the agreement known as 30x30 is to designate 30% of the world's land and sea masses as protected areas for biodiversity conservation by 2030.\"], \n    \"statements present in the answer but not found in the ground truth\": [\"This agreement aims to address the global biodiversity crisis and the impacts of climate change by preserving and restoring ecosystems, promoting sustainable land and ocean management practices, and safeguarding the habitats of numerous plant and animal species. The goal is to ensure the long-term health and resilience of the planet's natural resources and to support the well-being o

...

[chain/end] [1:chain:ragas evaluation > 6:chain:row 4 > 7:chain:answer_correctness > 9:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.7362132924074095
}
[chain/end] [1:chain:ragas evaluation > 6:chain:row 4 > 7:chain:answer_correctness] [34.66s] Exiting Chain run with output:
{
  "output": 0.18405332310185238
}
[llm/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:BedrockChat] [35.26s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are the extracted statements from the given question, answer and ground truth:\n\n{\"statements that are present in both the answer and the ground truth\": [], \n\n\"statements present in the answer but not found in the ground truth\": [\n\"The global implications of the USA Supreme Court ruling on abortion can be significant, as it sets a precedent for other countries and influences the global discourse on reproductive rights.\",\n\"Influence on other coun

Evaluating:  90%|█████████ | 18/20 [00:35<00:01,  1.56it/s]

[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 5:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.9434488306787865
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] [35.66s] Exiting Chain run with output:
{
  "output": 0.23586220766969662
}
[llm/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 5:llm:BedrockChat] [36.42s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are the extracted statements from the given question, answer and ground truth:\n\n{\"statements that are present in both the answer and the ground truth\": [\"The main contributors to GHG emissions and their role in global warming are primarily fossil fuel companies. These companies extract, produce, and sell fossil fuels, which when burned release carbon dioxide (CO2) and other GHGs into the atmosphere.\", \"The top contributors include Saudi Arabian Oil C

Evaluating: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it]

[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 6:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.7950714554701188
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness] [36.77s] Exiting Chain run with output:
{
  "output": 0.33513150023116606
}
[chain/end] [1:chain:ragas evaluation > 17:chain:row 15 > 18:chain:answer_correctness > 20:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.8731047819959566
}
[chain/end] [1:chain:ragas evaluation > 17:chain:row 15 > 18:chain:answer_correctness] [36.88s] Exiting Chain run with output:
{
  "output": 0.21827619549898916
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0] [36.94s] Exiting Chain run with output:
{
  "answer_correctness": 0.23586220766969662
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1] [36.94s] Exiting Chain run with output:
{
  "answer_correctness": 0.33513150023116606
}
[chain/end] [1:chain:raga

In [13]:
# Using Claude Instant
print(results_bedrock_bad)
results_bedrock_bad.to_pandas()

{'answer_correctness': 0.3721}


,question,ground_truth,answer,contexts,answer_correctness
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ...",0.235862
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[- Fossil fuel companies, whether state or pri...",0.335132
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The private companies responsible for the mos...,0.545200
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,[Amnesty International called on its vast netw...,0.571804
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,[Amnesty International recommends that the Spe...,0.184053
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,[Amnesty International has therefore created t...,0.413059
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[26. The Act raises serious questions about it...,0.411088
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,[- The victims of gross human rights violation...,0.444759
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,0.356503
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,0.382838


In [14]:
results_bedrock_bad_custom_dataset = evaluate(
    dataset['train'],
    llm=llm,
    embeddings=embeddings,
    metrics=[
        answer_correctness
    ]
)

[chain/start] [1:chain:ragas evaluation] Entering Chain run with input:
{}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Grey.",
  "contexts": [
    "My name is Grey"
  ]
}
[chain/start] [1:chain:ragas evaluation > 3:chain:row 1] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Matt.",
  "contexts": [
    "My name is Grey"
  ]
}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Grey.",
  "contexts": [
    "My name is Grey"
  ]
}
[llm/start] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:BedrockChat] Entering LLM run with input:
{
  "prompts": [
    "Human: Extract fol

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

[llm/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 5:llm:BedrockChat] [2.95s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are the extracted statements from the given questions, answers and ground truths:\n\nFor the first question:\n\n{\"statements that are present in both the answer and the ground truth\": [\"The sun's primary function is to provide light\"], \"statements present in the answer but not found in the ground truth\": [\"The sun is powered by nuclear fission\", \"similar to nuclear reactors on Earth\"], \"relevant statements found in the ground truth but omitted in the answer\": [\"The sun is powered by nuclear fusion, not fission\", \"In its core, hydrogen atoms fuse to form helium, releasing a tremendous amount of energy\", \"This energy provides heat and light, essential for life on Earth\", \"The sun's light plays a critical role in Earth's climate system\", \"The sun helps to drive the weather a

Evaluating:  50%|█████     | 1/2 [00:03<00:03,  3.23s/it]

[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 6:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.5399037286476214
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness] [3.23s] Exiting Chain run with output:
{
  "output": 0.30164259882857203
}
[llm/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:BedrockChat] [4.31s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are the extracted statements from the given questions, answers and ground truths:\n\nFor the first question:\n[\n  {\n    \"statements that are present in both the answer and the ground truth\": [\"The sun's primary function is to provide light\"], \n    \"statements present in the answer but not found in the ground truth\": [\"The sun is powered by nuclear fission\", \"similar to nuclear reactors on Earth\"],\n    \"relevant statements found in the ground tr

Evaluating: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]

[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 5:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.969163500586689
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] [4.63s] Exiting Chain run with output:
{
  "output": 0.4089575418133389
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0] [4.63s] Exiting Chain run with output:
{
  "answer_correctness": 0.4089575418133389
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1] [4.63s] Exiting Chain run with output:
{
  "answer_correctness": 0.30164259882857203
}
[chain/end] [1:chain:ragas evaluation] [4.63s] Exiting Chain run with output:
{
  "answer_correctness": 0.35530007032095545
}


In [15]:
# Using Claude Instant
print(results_bedrock_bad_custom_dataset)
results_bedrock_bad_custom_dataset.to_pandas()

{'answer_correctness': 0.3553}


,ground_truth,question,answer,contexts,answer_correctness
0,Your name is Grey,What is my name?,Your name is Grey.,[My name is Grey],0.408958
1,Your name is Grey,What is my name?,Your name is Matt.,[My name is Grey],0.301643


# Making Bedrock eval better

To get around this bad behavior, we need to use better prompts.  In this case, watching the logs, it looks like the `answer_correctness` metric has one main prompt that asks for a response and expects it in JSON. It gives a few shot example returning json, but does not actually give the instruction to return in that format, so we need to make that a little more clear.

[Here](https://docs.ragas.io/en/stable/howtos/applications/custom_prompts.html) are some instructions for overriding those prompts.  Unfortuately, the way this is set up, this is a destructive action that will mutate the `answer_correctness` metric to always use this prompt until your python kernal restarts.

In [16]:
# Save off the original istruction string so we have access later if we want it
# original_correctness_prompt_instruction = answer_correctness.correctness_prompt.instruction
original_correctness_prompt_instruction = "Extract following from given question and ground truth"

In [17]:
from ragas.llms.prompt import Prompt
from ragas.metrics import answer_correctness

new_instruction = f"""{original_correctness_prompt_instruction}.
Generate "Extracted Statements" similar to the {len(answer_correctness.correctness_prompt.examples)} examples I am about to give you.
Respond only in JSON format, just like the "Extracted Statements" in these two examples.
Do not say anything before or after the JSON output, or you will be penalized.
Only generate the data for the last question and ground truth provided."""

bedrock_correctness_prompt = Prompt(
    name="bedrock_correctness_prompt",
    instruction=new_instruction,
    examples=answer_correctness.correctness_prompt.examples,
    input_keys=answer_correctness.correctness_prompt.input_keys,
    output_key=answer_correctness.correctness_prompt.output_key,
    output_type=answer_correctness.correctness_prompt.output_type,
)

### Check what our new prompt looks like, then override the default prompt on the `answer_correctness` metric.  This is that destructive action I was talking about earlier.

In [18]:
print("OLD PROMPT:")
print(answer_correctness.correctness_prompt.to_string())

answer_correctness.correctness_prompt = bedrock_correctness_prompt

print("NEW PROMPT:")
print(answer_correctness.correctness_prompt.to_string())

OLD PROMPT:
Extract following from given question and ground truth

question: "What powers the sun and what is its primary function?"
answer: "The sun is powered by nuclear fission, similar to nuclear reactors on Earth, and its primary function is to provide light to the solar system."
ground_truth: "The sun is actually powered by nuclear fusion, not fission. In its core, hydrogen atoms fuse to form helium, releasing a tremendous amount of energy. This energy is what lights up the sun and provides heat and light, essential for life on Earth. The sun's light also plays a critical role in Earth's climate system and helps to drive the weather and ocean currents."
Extracted statements: [{{"statements that are present in both the answer and the ground truth": ["The sun's primary function is to provide light"], "statements present in the answer but not found in the ground truth": ["The sun is powered by nuclear fission", "similar to nuclear reactors on Earth"], "relevant statements found in 

### Now lets try the Bedrock eval again and see how it does.

In [19]:
results_bedrock = evaluate(
    amnesty_qa['eval'],
    llm=llm,
    embeddings=embeddings,
    metrics=[
        answer_correctness
    ]
)

[chain/start] [1:chain:ragas evaluation] Entering Chain run with input:
{}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0] Entering Chain run with input:
{
  "question": "What are the global implications of the USA Supreme Court ruling on abortion?",
  "ground_truth": "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion gui

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

[llm/end] [1:chain:ragas evaluation > 8:chain:row 6 > 9:chain:answer_correctness > 10:llm:BedrockChat] [1.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " [{\"statements that are present in both the answer and the ground truth\": [\"The right that guarantees access to comprehensive information about past human rights violations, including the identities of the perpetrators and the fate of the victims, as well as the circumstances surrounding the violations\"], \"statements present in the answer but not found in the ground truth\": [\"is the right to truth\"], \"relevant statements found in the ground truth but omitted in the answer\": [\"is the right to know the truth\"]}]",
        "generation_info": null,
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "k

Evaluating:  10%|█         | 2/20 [00:01<00:08,  2.02it/s]

[chain/end] [1:chain:ragas evaluation > 8:chain:row 6 > 9:chain:answer_correctness > 11:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.9776855897297182
}
[chain/end] [1:chain:ragas evaluation > 8:chain:row 6 > 9:chain:answer_correctness] [1.63s] Exiting Chain run with output:
{
  "output": 0.6194213974324295
}
[chain/end] [1:chain:ragas evaluation > 7:chain:row 5 > 8:chain:answer_correctness > 10:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.9855707464279734
}
[chain/end] [1:chain:ragas evaluation > 7:chain:row 5 > 8:chain:answer_correctness] [1.75s] Exiting Chain run with output:
{
  "output": 0.6213926866069933
}
[llm/end] [1:chain:ragas evaluation > 10:chain:row 8 > 11:chain:answer_correctness > 12:llm:BedrockChat] [1.76s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " [{\"statements that are present in both the answer and the ground truth\": [], \"statements present in the answer but not fou

...

[chain/end] [1:chain:ragas evaluation > 16:chain:row 14 > 17:chain:answer_correctness > 19:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.9016077284754098
}
[chain/end] [1:chain:ragas evaluation > 16:chain:row 14 > 17:chain:answer_correctness] [12.22s] Exiting Chain run with output:
{
  "output": 0.22540193211885245
}
[chain/end] [1:chain:ragas evaluation > 18:chain:row 16 > 19:chain:answer_correctness > 21:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.7880686099496743
}
[chain/end] [1:chain:ragas evaluation > 18:chain:row 16 > 19:chain:answer_correctness] [12.25s] Exiting Chain run with output:
{
  "output": 0.19701715248741858
}
[chain/end] [1:chain:ragas evaluation > 13:chain:row 11 > 14:chain:answer_correctness > 16:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.9103014902971095
}
[chain/end] [1:chain:ragas evaluation > 13:chain:row 11 > 14:chain:answer_correctness] [12.41s] Exiting Chain run with 

Evaluating:  95%|█████████▌| 19/20 [00:13<00:00,  1.97it/s]

[chain/end] [1:chain:ragas evaluation > 20:chain:row 18 > 21:chain:answer_correctness > 23:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.8825668929538506
}
[chain/end] [1:chain:ragas evaluation > 20:chain:row 18 > 21:chain:answer_correctness] [13.16s] Exiting Chain run with output:
{
  "output": 0.45141095400769343
}
[chain/end] [1:chain:ragas evaluation > 17:chain:row 15 > 18:chain:answer_correctness > 20:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.8731047819959566
}
[chain/end] [1:chain:ragas evaluation > 17:chain:row 15 > 18:chain:answer_correctness] [13.89s] Exiting Chain run with output:
{
  "output": 0.36827619549898916
}
[llm/end] [1:chain:ragas evaluation > 19:chain:row 17 > 20:chain:answer_correctness > 21:llm:BedrockChat] [13.98s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " [\n  {\n    \"statements that are present in both the answer and the ground truth\": [\n      \"Social medi

Evaluating: 100%|██████████| 20/20 [00:14<00:00,  1.40it/s]

[chain/end] [1:chain:ragas evaluation > 19:chain:row 17 > 20:chain:answer_correctness > 22:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.9033563104129538
}
[chain/end] [1:chain:ragas evaluation > 19:chain:row 17 > 20:chain:answer_correctness] [14.32s] Exiting Chain run with output:
{
  "output": 0.3622027139668748
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0] [14.93s] Exiting Chain run with output:
{
  "answer_correctness": 0.4123327959049907
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1] [14.93s] Exiting Chain run with output:
{
  "answer_correctness": 0.3237678638675297
}
[chain/end] [1:chain:ragas evaluation > 4:chain:row 2] [14.93s] Exiting Chain run with output:
{
  "answer_correctness": 0.4545409081201908
}
[chain/end] [1:chain:ragas evaluation > 5:chain:row 3] [14.93s] Exiting Chain run with output:
{
  "answer_correctness": 0.19680423645077616
}
[chain/end] [1:chain:ragas evaluation > 6:chain:row 4] [14.93s] Exiting Chain run wi

In [20]:
# Using Claude Instant
print(results_bedrock)
results_bedrock.to_pandas()

{'answer_correctness': 0.3519}


,question,ground_truth,answer,contexts,answer_correctness
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ...",0.412333
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[- Fossil fuel companies, whether state or pri...",0.323768
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The private companies responsible for the mos...,0.454541
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,[Amnesty International called on its vast netw...,0.196804
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,[Amnesty International recommends that the Spe...,0.184053
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,[Amnesty International has therefore created t...,0.621393
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[26. The Act raises serious questions about it...,0.619421
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,[- The victims of gross human rights violation...,0.444759
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,0.189836
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,0.216172


In [21]:
results_bedrock_custom_dataset = evaluate(
    dataset['train'],
    llm=llm,
    embeddings=embeddings,
    metrics=[
        answer_correctness
    ]
)

[chain/start] [1:chain:ragas evaluation] Entering Chain run with input:
{}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Grey.",
  "contexts": [
    "My name is Grey"
  ]
}
[chain/start] [1:chain:ragas evaluation > 3:chain:row 1] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Matt.",
  "contexts": [
    "My name is Grey"
  ]
}
[chain/start] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] Entering Chain run with input:
{
  "ground_truth": "Your name is Grey",
  "question": "What is my name?",
  "answer": "Your name is Grey.",
  "contexts": [
    "My name is Grey"
  ]
}
[llm/start] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:BedrockChat] Entering LLM run with input:
{
  "prompts": [
    "Human: Extract fol

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

[llm/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 4:llm:BedrockChat] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " [{\"statements that are present in both the answer and the ground truth\": [\"Your name is Grey\"], \"statements present in the answer but not found in the ground truth\": [], \"relevant statements found in the ground truth but omitted in the answer\": []}]",
        "generation_info": null,
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": " [{\"statements that are present in both the answer and the ground truth\": [\"Your name is Grey\"], \"statements present in the answer but not found in the ground truth\": [], \"relevant statements found in the ground truth but omitted in t

Evaluating: 100%|██████████| 2/2 [00:01<00:00,  1.70it/s]

[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness > 5:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.969163500586689
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0 > 3:chain:answer_correctness] [1.07s] Exiting Chain run with output:
{
  "output": 0.9922908751466722
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness > 6:chain:answer_similarity] s] Exiting Chain run with output:
{
  "output": 0.5399037286476214
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1 > 4:chain:answer_correctness] [1.19s] Exiting Chain run with output:
{
  "output": 0.13497593216190534
}
[chain/end] [1:chain:ragas evaluation > 2:chain:row 0] [1.19s] Exiting Chain run with output:
{
  "answer_correctness": 0.9922908751466722
}
[chain/end] [1:chain:ragas evaluation > 3:chain:row 1] [1.19s] Exiting Chain run with output:
{
  "answer_correctness": 0.13497593216190534
}
[chain/end] [1:chain:ragas evaluation] 

In [22]:
# Using Claude Instant
print(results_bedrock_custom_dataset)
results_bedrock_custom_dataset.to_pandas()

{'answer_correctness': 0.5636}


,ground_truth,question,answer,contexts,answer_correctness
0,Your name is Grey,What is my name?,Your name is Grey.,[My name is Grey],0.992291
1,Your name is Grey,What is my name?,Your name is Matt.,[My name is Grey],0.134976


# Comparing the results

In [23]:
def results_string(dataset):
    return f"""{dataset["answer_correctness"]}  (Invalid Records: {dataset.to_pandas()['answer_correctness'].isnull().sum()}/{len(dataset.dataset)})"""

print("Premade Dataset:")
print(f"OpenAI: {results_string(results_openai)}")
print(f"Bad Bedrock: {results_string(results_bedrock_bad)}")
print(f"Hopefully Better Bedrock: {results_string(results_bedrock)}")
print("")
print("Custom Dataset (Should be around .5 ideally):")
print(f"OpenAI: {results_string(results_openai_custom_dataset)}")
print(f"Bad Bedrock: {results_string(results_bedrock_bad_custom_dataset)}")
print(f"Hopefully Better Bedrock: {results_string(results_bedrock_custom_dataset)}")

Premade Dataset:
OpenAI: 0.4459275307294037  (Invalid Records: 0/20)
Bad Bedrock: 0.3720638482151994  (Invalid Records: 0/20)
Hopefully Better Bedrock: 0.3518763886159751  (Invalid Records: 0/20)

Custom Dataset (Should be around .5 ideally):
OpenAI: 0.6062954601885172  (Invalid Records: 0/2)
Bad Bedrock: 0.35530007032095545  (Invalid Records: 0/2)
Hopefully Better Bedrock: 0.5636334036542888  (Invalid Records: 0/2)
